In [88]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

%matplotlib inline

In [89]:
@widgets.interact
def square(x=5):
    print(x)

interactive(children=(IntSlider(value=5, description='x', max=15, min=-5), Output()), _dom_classes=('widget-in…

In [90]:
@widgets.interact_manual(color=['blue', 'red', 'green'], lw=(1., 10.))
def plot(freq=1., color='blue', lw=2, grid=True):
    t = np.linspace(-1., +1., 1000)
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    ax.plot(t, np.sin(2 * np.pi * freq * t), lw=lw, color=color)
    ax.grid(grid)

interactive(children=(FloatSlider(value=1.0, description='freq', max=3.0, min=-1.0), Dropdown(description='col…

In [91]:
rho = 1.225 # [kg/m3]
f = 1e-4  # [1/s]
p_0 = 1000  # [hPa]

In [92]:
def generate_gaussian_pressure_system(Nx, Ny, Lx, Ly, p_c, sigma):
    x, y = np.meshgrid(np.linspace(-Lx/2, Lx/2, Nx), np.linspace(-Ly/2, Ly/2, Ny))
    d = np.sqrt(x*x + y*y)
    mu = 0
    p = p_0 + (p_c - p_0)*np.exp(-((d - mu)**2 / ( 2.0 * sigma**2 )))
    return x, y, p

In [93]:
def calculate_geostrophic_velocity(x, y, p):
    u, v = np.zeros(p.shape), np.zeros(p.shape)
    dx, dy = x[0, 1] - x[0, 0], y[1, 0] - y[0, 0]
    delta_x_p, delta_y_p = np.gradient(p)
    u = - (1 / (f*rho)) * delta_x_p / dy
    v =   (1 / (f*rho)) * delta_y_p / dx
    return u, v

In [96]:
@widgets.interact(p_c=(950, 1030), sigma=(10e3, 30e3), hemisphere=["Northern", "Southern"])
def plot(p_c=980, sigma=20e3, hemisphere="Northern"):
    Nx, Ny = 100, 100
    Lx, Ly = 100e3, 100e3
    
    x, y, p = generate_gaussian_pressure_system(Nx, Ny, Lx, Ly, p_c, sigma)
    u, v  = calculate_geostrophic_velocity(x, y, p)
    
    fig = plt.figure(figsize=(10, 20), dpi=100)
    
    ax1 = plt.subplot(311)
    plt.contourf(x/1000, y/1000, p, vmin=950, vmax=1050, cmap="seismic")
    plt.title("Pressure field")
    plt.xlabel("x (km)")
    plt.ylabel("y (km)")
    plt.colorbar()
    
    ax2 = plt.subplot(312)
    speed = u*u + v*v
    plt.contourf(x, y, speed, cmap="plasma")
    plt.title("Velocity magnitude field")
    plt.xlabel("x (m)")
    plt.ylabel("y (m)")
    plt.colorbar()
    
    plt.quiver(x[::15], y[::15], u[::15], v[::15])
    # lw = 5*speed / speed.max()
    # plt.streamplot(x, y, u, v, linewidth=lw)

interactive(children=(IntSlider(value=980, description='p_c', max=1030, min=950), FloatSlider(value=20000.0, d…